# Chatbots and Retrieval Systems

Course "**Programmazione di Applicazioni Data Intensive**" - 22th of February, 2024

This notebook was created by:
- PhD. student Lorenzo Molfetta
- Prof. Gianluca Moro
- PhD. student Giacomo Frisoni

__CONTACTS__:

If you have any doubt, contact me at: lorenzo.molfetta@unibo.it

__INDEX__:

In this notebook we will delve into the performance of generative LLMs applied to highly technical  . We will cover ...

The notebook is divided into the following sections:
1. [Download data](#utils)
2. [RAG - Talk-to-Text - Retrieval with Llama-index](#talk_2)
3. [Implementing a Retrieval-enhanced Chatbot](#chat)

__HOW TO RUN THIS NOTEBOOK__

The following code will explore different topics using several approaches and technologies. To give you the possibility to run all of this on your (__FREE !!!__) Colab account, we have prepared package versions and model configurations that can run without major memory issues on the standard Tesla T4 GPU (16 GB) and default RAM (12 GB).


<br><br>
Before starting and in the case the runtime crashes, follow these basic instructions:
- change runtime type and enable the GPU usage;
- every time you start a new session, there's no need to install packages you've already downloaded. Just follow the workflow of each section and import the functions you need;
- download data from GitHub if needed.

## 1. Download data
<a id='utils'></a>

You can find useful data for the following experiments in this repository: https://github.com/LorMolf/Seminar-Chatbot.git

In [1]:
! git clone https://github.com/LorMolf/Seminar-Chatbot.git

Cloning into 'Seminar-Chatbot'...


In [2]:
! unzip Seminar-Chatbot/friends_scripts.zip -d . 1>/dev/null

The system cannot find the path specified.



## 2. RAG - Talk-to-Text - Retrieval with Llama-index
<a id='talk_2'></a>

`LlamaIndex` is a powerful tool that acts as a bridge between your custom data and LLMs. It makes your data more accessible and usable, paving the way for creating powerful custom LLM applications and workflows.

The main features of LlamaIndex can be summarised as:
- __Data Ingestion__: LlamaIndex helps ingest data, which means getting the data from its source into the system. It offers data connectors for a variety of data sources and formats, such as APIs, PDFs, documents, and SQL databases;
- __Data Structuring__: it helps in structuring the data, which means organising information simply for the model to access. This is done by parsing the documents into nodes, which are chunks of text. The ‘data indexes’ are the organised librarians, arranging your data neatly to be easily accessible.
- __Data Retrieval__: A retrieval support helps models find and fetch the right pieces of data when needed. An index is constructed so LlamaIndex can quickly retrieve the relevant data when we query the documents. The index can further be stored in different ways.
- __Integration__: a simplified integration makes melding the data with various application frameworks easier. The “engines” are the translators (LLMs), enabling interaction with your data using natural language and ultimately creating applications and workflows.

In a few words, LlamaIndex is streamlined for the workflow described above. It provides a query interface that accepts any input prompt over your data and returns a knowledge-augmented response.

The core engine of the retrieval-based query system is the `ServiceContext` object. It is the main actor in the architecture which is able to coordinate all the differnt agents in the retrieve-then-predict chain.
Specifically, a ServiceContext encapsulate:
- `model`: the language model for inference;
- `embeddig-model`: an encoder supporting the retrieval of external information by generating vector representations of text used for a similarity search;
- `text-splitter`: an handler for indexing the external corpus based on a given cardinality for the chunks data is divided into;
- `prompt-helper`: a prompting system in charge of parsing the input query with the proper prompting instructions. It provides utility for “repacking” text chunks (retrieved from index) to maximally make use of the available context window (and thereby reducing the number of LLM calls needed), or truncating them so that they fit in a single LLM call.

### Install libraries
To make LlamaIndex work on Colab, we need to modify some environment variables and install specific libraries' version. If you've ran the previous experiments, we suggest you to delete the current runtime and restart a new one.

In [3]:
! export CMAKE_ARGS="-DLLAMA_OPENBLAS=on"
! export FORCE_CMAKE="1"

! CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install --upgrade --force-reinstall llama-cpp-python numpy==1.25

'export' is not recognized as an internal or external command,
operable program or batch file.
'export' is not recognized as an internal or external command,
operable program or batch file.
'CMAKE_ARGS' is not recognized as an internal or external command,
operable program or batch file.


Before proceeding with the next downloads, we need to make these liberary version changes effective. So we need to __RESTART THE RUNTIME__ (do NOT delete it though).

In [4]:
# AFTER RUNTIME RESTART
!pip install llama-index==0.9.10
!pip install transformers datasets sentence-transformers
!pip install langchain
!pip install nest_asyncio

^C


__TL;DR__: Run the following cell every time you restart the runtime to enable asynchronous thread pooling.

<br>

_Asynchronous threads_ :
When the LlamaIndex library is used to launch a batched inference or a retrieval-based generation of the dataset, multiple threads are scheduled asynchronously to enhance performances and optimize latency and throughput. However, Colab does not support this behaviour natively. We thus need to enable asynchronous pooling to make this library work properly. Using the `nest_asyncio` package, we can change the environment setting for this purpose.



In [ ]:
from contextlib import contextmanager,redirect_stderr,redirect_stdout
from os import devnull
import nest_asyncio

# ENABLE ASYNCH
nest_asyncio.apply()

# SUPPRESS WARNINGS
@contextmanager
def suppress_stdout_stderr():
    """A context manager that redirects stdout and stderr to devnull"""
    with open(devnull, 'w') as fnull:
        with redirect_stderr(fnull) as err, redirect_stdout(fnull) as out:
            yield (err, out)

@contextmanager
def suppress_stderr():
    """A context manager that redirects stderr to devnull"""
    with open(devnull, 'w') as fnull:
        with redirect_stderr(fnull) as err:
            yield err

Now we can import everything we need.

In [ ]:
# General purpose libraries
import logging
import sys
import pandas as pd
import json
import torch
from tqdm import tqdm
import os

from IPython.display import Markdown, display

# >> Llama-Index
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
    LLMPredictor,
    Response,
    PromptHelper,
    download_loader
)

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index.node_parser import SentenceSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.prompts.base import PromptTemplate
from llama_index.response.notebook_utils import display_source_node


device = "cuda" if torch.cuda.device_count() > 0 else "cpu"

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

For the next experiments we are going to use the `Zephyr-7B-Beta` and `Cerbero-7B` models. If you either haven't donwloaded them before or deleted the runtime, install and save them as follows.

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------------------
## >> Zephyr-7b
zephyr_model_name = "TheBloke/zephyr-7B-beta-GGUF"
zephyr_model_file = "zephyr-7b-beta.Q4_K_M.gguf"
! huggingface-cli download TheBloke/zephyr-7B-beta-GGUF zephyr-7b-beta.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False


# -------------------------------------------------------------------------------------------------------------------------------------
## >> Cerbero-7b
cerbero_model_name = "galatolo/cerbero-7b-gguf"
cerbero_model_file = "ggml-model-Q4_K.gguf"
! huggingface-cli download galatolo/cerbero-7b-gguf ggml-model-Q4_K.gguf --local-dir . --local-dir-use-symlinks False

### Retrieval over external data sources

Before diving into the developing of an inference engine for question answering supported by a first phase of fetching of useful information from an external corpus, we want to test the ability of the retrieval and compare the performances of different encoder models.

__INDEXING A PDF - "Codice degli Appalti 2023"__:

This section will show how to deal with knowledge from the Italian Procurement Code ("Codice degli Appalti"). The document legislates on the public tenders by setting a complex ruling system acting on the funding access and the contract requirements during the entire procurement procedure. This kind of text has a highly technical syntax and thus results in a challenging benchmark for the model to prove its abilities. We provide you with the latest version of the Procurement Code modified in June 2023. In so doing, we are sure the pre-trained models have no previous knowledge on this matter and desperately need external information on which to base their predictions.

Here's a small dataset of question-answer examples on this particular version of the Procurement Code.

In [ ]:
ca_dataset = pd.read_csv('Seminar-Chatbot/CdA-mininterno-quiz_dataset.csv')

LlamaIndex provides a set of high-level tools for parsing any data format. You can ingest information from text docuents, directories or more structured formats such as PDF. Without installing any external libraries, we can use the internal features of LlamaIndex to load and split the "Codice degi Appalti" PDF file into nodes, which are the atomic object used to index and traverse information.

In [ ]:
# Use a PDF Reader to parse the document
PDFReader = download_loader("PDFReader")
loader = PDFReader()
documents = loader.load_data(file='Seminar-Chatbot/ca.pdf')

# Create nodes
node_parser = SentenceSplitter(chunk_size=512)
nodes = node_parser.get_nodes_from_documents(documents)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


__LLM Engine__

Depending on the nature of the pretrained architecture's format, we can instantiate the query model using differnt classes provided by the LlamaIndex library. While you can import models directly from HuggingFace with the `HuggingFaceLLM` class, using `LlamaCPP` we can exploit the efficiency of GGUF format and C-Transformers.

<br>
For this part, we are going to use the Cerbero model.

In [ ]:
context_window = 8192
max_output_tokens = 1024

In [ ]:
# ---------------------------------------------------------------------------
## >> PROMPT HELPER

prompt_helper_config = {
    'context_window' : context_window,
    'num_output' : max_output_tokens,
    'chunk_overlap_ratio' : 0 # The percentage token amount that each chunk should overlap.
}
prompt_helper = PromptHelper(**prompt_helper_config)

# ---------------------------------------------------------------------------
## >> INFERENCE MODEL

custom_model_config = {
    'verbose' : True,
    'temperature' : 0.2,
    'max_new_tokens' : max_output_tokens,
    'model_kwargs' : {
        "n_gpu_layers": 50, # map layer to GPU
        "device" : device,
        "stop" : ['[|Umano|]', '[|AI|]','[end of text]'],
    }
}

llm = LlamaCPP(
    model_path=cerbero_model_file,
    context_window=context_window,
    **custom_model_config
)

# Incapsulate the model into the LLMPredictor object
# before creating the Service Context
llm_predictor = LLMPredictor(llm=llm)

#### __HOW TO RETRIEVE__:

Retrieval using similarity search is an expensive operation for an extensive search space as the number of comparisons needed to estimate the top-$k$ resources grows factorially with the number of source documents. Many approximate-search engines available on the market (e.g. FAISS) rely on the definition of a precomputed structure of documents that is easier to traverse for the similarity engine.

In LlamaIndex, the indexing of document nodes is charged to the VectorIndex, which configures the data node structure and yields efficient information retrieval. The retriever model is instantiated directly from this Index and can be further configured to retrieve more than one document based on the similarity with the input query.

In [ ]:
# Create Service Context Manager
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor,
                                               embed_model='local',
                                               prompt_helper=prompt_helper)

# Initilize structure of information nodes
vector_index = VectorStoreIndex(nodes, service_context=service_context)

# Define retriever that is able to select 3 different document chunks useful for the answer generation
retriever = vector_index.as_retriever(similarity_top_k=3)

In [ ]:
nodes = []
for query in tqdm(ca_dataset['Domanda']):
  retr_nodes = retriever.retrieve(query)
  nodes.append(retr_nodes)

100%|██████████| 451/451 [00:39<00:00, 11.39it/s]


In [ ]:
n_nodes_x_q = len(nodes[0])

retr_nodes = {
    'query' : [],
    'page_labels' : []
}
for i in range(n_nodes_x_q): retr_nodes[f'source_{i+1}'] = []

for i, query in enumerate(tqdm(ca_dataset['Domanda'])):
  retr_nodes['query'].append(query)
  p_labels = []
  for node_i in range(n_nodes_x_q):
    nn = nodes[i][node_i]
    retr_nodes[f'source_{node_i+1}'].append(nn.text)
    p_labels.append(nn.metadata['page_label'])
  retr_nodes['page_labels'].append(p_labels)

100%|██████████| 451/451 [00:00<00:00, 71199.60it/s]


In [ ]:
retrieval_dataset = pd.DataFrame.from_dict(retr_nodes)
retrieval_dataset.head()

,query,page_labels,source_1,source_2,source_3
0,A chi è demandata la valutazione dell'offerta ...,"[238, 80, 131]",16. In caso di procedura competitiva con nego...,3. Può essere utilizzato il criterio del minor...,2. L'aggiudicazione avviene secondo il criteri...
1,A cosa è tenuto l'operatore economico che part...,"[75, 75, 257]","Non sono sanabili le omissioni, \ninesattezze ...",4. Fino al giorno fissato per la loro apertura...,4. Qualora l'operatore economico non risponda ...
2,A fronte dell'iniziativa di una stazione appal...,"[143, 255, 255]",Articolo 219. - Scioglimento del collegio con...,"6. Le SOA trasmettono all'ANAC, entro quindici...",4. L'ANAC provvede periodicamente alla verific...
3,A norma del Codice dei contratti pubblici e de...,"[349, 20, 11]",Tale collegamento garantisce un accesso \nimme...,2. Le stazioni appaltanti e gli enti concedent...,LIBRO I | PARTE I - DEI PRINCIP I \nArticolo ...
4,A norma delle Legge 136 del 2010 chi attribuis...,"[135, 49, 49]",LIBRO V | PARTE I -DEL CONTENZIOSO \nArticol...,"267, no nché loro consorzi e associazioni, e g...",In tale caso l’Autorità nazionale anticorru-\n...


#### __TESTING RETRIEVAL's ENCODER EFFICIENCY__:

Tuning the configuration of the retrieval model is pivotal for fetching data at the same time consistent with the input query and helpful for the answer generation. This can be done by adjusting the size of chunks for the corpus indexing - depending on the specific needs of the application scenario - and by employing ad-hoc and in-domain expert encoder models. The `BAAI/bge` is one of the top-ranked encoder famility with outstading performances in a vast variety of tasks. We are now going to test two different versions of these encoders and evaluate them based on the content reported in the retrieved paragraphs.

The precision of a retrieval system can be gauged using many different metrics (e.g. _Hit-Rate_, _Mean Reciprocal Rank_, ...). When gold data is not available - as in our case - we can evaluate the coherence of the retrieved information based on the overlap of their conent with the input question and expected answer. It is indeed extremely important that selected information are equally in line with the query and useful for the derivation of the answer. To do so, we check the presence of key information for the answer in each retrieved node. Without any particular semantic parsing process, we can simply select the meaningful terms in the answer neglecting stop words.
<br>

_Discounted Cumulative Gain_ (DCG):

DCG measures the usefulness, or gain, of a document based on its position in the result list. The gain is accumulated from the top of the result list to the bottom, with the gain of each result discounted at lower ranks.
The basline assumptions are that highly relevant documents are more useful when appearing earlier in a search engine result list (have higher ranks). DCG is a refinement of a simpler measure, Cumulative Gain (CG), which is the sum of the graded relevance values of all results in a search result list, which does not take into account the rank (position) of a result in the result list.


The premise of DCG is that highly relevant documents appearing lower in a search result list should be penalized as the graded relevance value is reduced logarithmically proportional to the position of the result:
$$
\text{DCG}_p = \sum_{i=1}^p \frac{2^{\rho_i} - 1}{\log_2 (i+1)}
$$
with $\rho_i$ being the relevance score assigned to the $i$-th document $d_i$.

In our experiment we set the relevance score $\rho_i$ to the number of (non-stop) words in common with the gold label, normalized by the sentence length. For instance, given the desired answer $\hat{a}$ and the list of relevant terms $\pi_i$ in the retrieved document $d_i$
$$
\rho_i = \frac{ \sum_{j=0}^{\lvert \pi_i \rvert } \left ( \pi_i [ j ] \in \pi_{\hat{a}} \right )}{ \lvert \pi_i \rvert }
$$
where $\pi_{\hat{a}}$ is the list of relevant terms contained in the golf answer.


We compute such measure for both the question and answer, respectively to verify the coherence of the retrieved passages and their level of informativeness to derive the answer.

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import re
import numpy as np
from typing import List

nltk.download('stopwords')
nltk.download('punkt')

def __word_highlights(sentence,
                      language : str = 'english'):
    """
    Select the "non-trivial" words from the input sentence by removing
    stop words.
    """

    # Handle new lines with dash
    sentence = re.sub('-\n','',sentence)

    # Remove punctuation
    sentence = sentence.translate(str.maketrans('', '', string.punctuation))

    # Tokenize the sentence into words
    words = word_tokenize(sentence)

    # Remove stopwords
    stop_words = set(stopwords.words(language))
    filtered_words = [word for word in words if word.lower() not in stop_words]

    if len(filtered_words) == 0:
      filtered_words = words

    return filtered_words


def __evaluate_resource(query : str,
                        answer : str,
                        resource : str):
  """
  Evaluate a retrieve resource based on the input question and
  the expected answer.
  """

  metrics = {
      'query_resource_overlap' : 0.0,
      'answer_resource_overlap' : 0.0
  }

  parsed_q = __word_highlights(query, language='italian')
  parsed_a = __word_highlights(answer, language='italian')
  parsed_r = __word_highlights(resource, language='italian')

  q_precision = np.count_nonzero([q_word in parsed_r for q_word in parsed_q])
  a_precision = np.count_nonzero([a_word in parsed_r for a_word in parsed_a])

  metrics['query_resource_overlap'] = q_precision / len(parsed_q)
  metrics['answer_resource_overlap'] = a_precision / len(parsed_a)

  return metrics

def __compute_dcg(oredered_scores : List):
  """
  Compute Discounted Cumulative Score
  """
  get_score = lambda i,score : (2**(score) - 1) / (np.log2(i+1))
  res = sum([get_score(i+1,s) for  i,s in enumerate(oredered_scores)])

  return res

def evaluate_retriever(query : str,
                       answer : str,
                       resource_list : List):
  """
  Evaluate the set of retrieved nodes for the input query.
  """

  overall_metrics = {
      'query_resource_overlap' : [],
      'answer_resource_overlap' : [],
      'dcg_query' : 0.0,
      'dcg_answer' : 0.0
  }

  for node in resource_list:
    # Evaluate coherence of each node
    res_txt = node.text
    stats = __evaluate_resource(query, answer, res_txt)
    for k in stats: overall_metrics[k].append(stats[k])


  overall_metrics['dcg_query'] = __compute_dcg(overall_metrics['query_resource_overlap'])
  overall_metrics['dcg_answer'] = __compute_dcg(overall_metrics['answer_resource_overlap'])

  for k in ['query_resource_overlap', 'answer_resource_overlap']:
    _sum_val = sum(overall_metrics[k])
    overall_metrics[k] = _sum_val / len(resource_list)

  return overall_metrics


def evaluate_retriever_handler(retriever, ca_dataset):
    """
    Evaluate retrieval statistics for each question in the input dataset.
    """

    retriever_stats = {
      'query' : [],
      'answer' : [],
      'query_resource_overlap' : [],
      'answer_resource_overlap' : [],
      'dcg_query' : [],
      'dcg_answer' : []
    }

    for query, answer in tqdm(zip(ca_dataset['Domanda'], ca_dataset['Risposta']), total=len(ca_dataset['Domanda'])):
      retrieved_nodes = retriever.retrieve(query)
      metrics = evaluate_retriever(query, answer, retrieved_nodes)

      for k in metrics: retriever_stats[k].append(metrics[k])

      retriever_stats['query'].append(query)
      retriever_stats['answer'].append(answer)

    return retriever_stats

def compute_avg_stats(retriever_stats):
  """
  Compute and print average retrieval statistics.
  """
  avg_stats = {}
  for k in list(set(retriever_stats.keys()) - set(['query','answer'])):
    avg_stats[k] = np.mean(retriever_stats[k])

  print("AVERAGE STATISTICS")
  for k in avg_stats:
    print(f"\t - {k.upper()} : {round(avg_stats[k],4)}")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


##### ENCODER MODEL <-- BAAI/bge-base-en-v1.5

In [ ]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name='BAAI/bge-base-en-v1.5'))

In [ ]:
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor,
                                               embed_model=embed_model,
                                               prompt_helper=prompt_helper)


vector_index = VectorStoreIndex(nodes, service_context=service_context)
retriever = vector_index.as_retriever(similarity_top_k=3)

Let's visualize the result of the retrieval for a question in the dataset.

In [ ]:
qa_ids = 10
query = ca_dataset['Domanda'][qa_ids]
answer = ca_dataset['Risposta'][qa_ids]

retrieved_nodes = retriever.retrieve(query)

qa = """
### QUERY:
{}

### ANSWER:
{}

"""
print(qa.format(query, answer))

print('#'*100)

for node in retrieved_nodes:
  print('-'*50)
  display_source_node(node, source_length=1000)

print('\n'*3+'#'*100+'\nSTATISTICS:\n')
metrics = evaluate_retriever(query, answer, retrieved_nodes)
for k in metrics: print(f'\t - {k.upper()} : {round(metrics[k],4)}')


### QUERY:
A norma di quanto dispone il Codice dei contratti pubblici in quale momento le stazioni appaltanti individuano i criteri di selezione delle offerte?

### ANSWER:
Prima dell'avvio delle procedure di affidamento


####################################################################################################
--------------------------------------------------


**Node ID:** a4b5b82a-82ed-4492-9eaa-6d58f1175249<br>**Similarity:** 0.7777024877987047<br>**Text:** Articolo 2.  - Definizioni dei contratti.  
1. Nel codice si intende per:  
a) «contratti » o « contratti pubblici », i contratti, anche diversi da appalti e 
concessioni, conclusi da una stazione appaltante o da un ente conce-
dente;  
b) «contratti di appalto » o « appalti pubblici », i contratti a titolo oneroso 
stipulati per iscritto tra uno o più operatori economici e una o più sta-
zioni appaltanti e aventi per oggetto l’esecuzion e di lavori, la fornitura 
di beni o la prestazione di servizi;  
c) «contratti di concessione » o « concessioni », i contratti a titolo oneroso 
stipulati per iscritto a pena di nullità in virtù dei quali una o più ammini-
strazioni aggiudicatrici o uno o più enti aggiudicatori affidano<br>

--------------------------------------------------


**Node ID:** 48961897-86d1-47d2-a2f4-d1f1ad474c26<br>**Similarity:** 0.7553046950911909<br>**Text:** I contratti conclusi con l’accettazione di tali ordinativi non sono sot-
toposti al parere di congruità economica. Ove previsto nel bando di gara, le convenzioni 
possono essere st ipulate con una o più imprese alle stesse condizioni contrattuali proposte 
dal miglior offerente. Ove previsto nel bando di gara, le convenzioni possono essere stipulate 
per specifiche categorie di amministrazioni ovvero per specifici ambiti territoriali. Il quarto 
periodo si applica anche agli accordi quadro stipulati dalla Consip S.p.A. ai sensi dell’articolo<br>

--------------------------------------------------


**Node ID:** 6f16556c-611e-4f19-b2ed-8d8a25e32657<br>**Similarity:** 0.752276933911786<br>**Text:** 2. La stazione appaltante assicura l’opportuna pubblicità dell’atti-
vità di esplorazione del mercato, scegliendo gli strumenti più idonei 
in ragione della rilevanza del contratto per il settore merceologico 
di riferimento e della sua contendibilità. A tal fin e la stazione appal-
tante pubblica un avviso sul suo sito istituzionale e sulla Banca dati 
nazionale dei contratti pubblici dell’ANAC. La durata della pubbli-
cazione è stabilita in ragione della rilevanza del contratto, per un 
periodo minimo identificabile i n quindici giorni, salva la riduzione 
del suddetto termine per motivate ragioni di urgenza a non meno di 
cinque giorni.  
3. L’avviso di avvio dell’indagine di mercato indica il valore dell’af-
fidamento, gli elementi essenziali del contratto, i requisiti di i do-
neità professionale, i requisiti minimi di capacità economica e finan-
ziaria e le capacità tecniche e professionali richieste ai fini della par-
tecipazione, il numero minimo ed eventualmente massimo di ...<br>




####################################################################################################
STATISTICS:

	 - QUERY_RESOURCE_OVERLAP : 0.25
	 - ANSWER_RESOURCE_OVERLAP : 0.0833
	 - DCG_QUERY : 0.4274
	 - DCG_ANSWER : 0.0946


In [ ]:
retriever_stats = evaluate_retriever_handler(retriever, ca_dataset)

100%|██████████| 451/451 [01:50<00:00,  4.09it/s]


In [ ]:
retrieval_dataset = pd.DataFrame.from_dict(retriever_stats)
retrieval_dataset.head()

,query,answer,query_resource_overlap,answer_resource_overlap,dcg_query,dcg_answer
0,A chi è demandata la valutazione dell'offerta ...,A una commissione giudicatrice,0.466667,0.000000,0.817908,0.000000
1,A cosa è tenuto l'operatore economico che part...,Deve usare conti correnti postali o bancari de...,0.272727,0.000000,0.436164,0.000000
2,A fronte dell'iniziativa di una stazione appal...,vincolante,0.155556,0.000000,0.209896,0.000000
3,A norma del Codice dei contratti pubblici e de...,"No, mai",0.205128,0.000000,0.339168,0.000000
4,A norma delle Legge 136 del 2010 chi attribuis...,L'Autorità di vigilanza sui contratti pubblici...,0.074074,0.433333,0.130572,0.885913


In [ ]:
# BAAI/bge-base-en-v1.5
compute_avg_stats(retriever_stats)

AVERAGE STATISTICS
	 - QUERY_RESOURCE_OVERLAP : 0.2568
	 - DCG_ANSWER : 0.2375
	 - ANSWER_RESOURCE_OVERLAP : 0.1305
	 - DCG_QUERY : 0.4523


##### ENCODER MODEL <-- BAI/bge-small-en

In [ ]:
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor,
                                               embed_model='local', # BAI/bge-small-en default implementation
                                               prompt_helper=prompt_helper)
vector_index = VectorStoreIndex(nodes,
                                service_context=service_context)

retriever = vector_index.as_retriever(similarity_top_k=3)

In [ ]:
local_retriever_stats = evaluate_retriever_handler(retriever, ca_dataset)

local_retrieval_dataset = pd.DataFrame.from_dict(local_retriever_stats)
local_retrieval_dataset.to_csv('ca_local_retrieval_stats.csv')
local_retrieval_dataset.head()

100%|██████████| 451/451 [00:54<00:00,  8.33it/s]


,query,answer,query_resource_overlap,answer_resource_overlap,dcg_query,dcg_answer
0,A chi è demandata la valutazione dell'offerta ...,A una commissione giudicatrice,0.433333,0.000000,0.699591,0.000000
1,A cosa è tenuto l'operatore economico che part...,Deve usare conti correnti postali o bancari de...,0.242424,0.000000,0.396876,0.000000
2,A fronte dell'iniziativa di una stazione appal...,vincolante,0.266667,0.000000,0.564735,0.000000
3,A norma del Codice dei contratti pubblici e de...,"No, mai",0.076923,0.000000,0.098382,0.000000
4,A norma delle Legge 136 del 2010 chi attribuis...,L'Autorità di vigilanza sui contratti pubblici...,0.370370,0.333333,0.654742,0.623604


In [ ]:
# BAI/bge-small-en
compute_avg_stats(local_retriever_stats)

AVERAGE STATISTICS
	 - ANSWER_RESOURCE_OVERLAP : 0.161
	 - DCG_ANSWER : 0.3024
	 - QUERY_RESOURCE_OVERLAP : 0.2863
	 - DCG_QUERY : 0.5116


The obtained results show a slight difference between the two retrieval encoders regarding the query-resource similarity. However, the `BAI/bge-small-en` provide a more significant improvement in the relevance of retrieved documents with respect to the expected answer. If we had to choose among the two options, even if the BAAI/bge-base-en-v1.5 encoder were to perform better in query-resource precision, we would still need to use the BAI/bge-small-en as it provides more informative information for the model to derive a faithful and accurate answer.


__Encoder Model__ | __Query-Resource Overlap__ | __DCG-Query__ | __Answer-Resource Overlap__  | __DCG-Answer__
:---: | :---: | :---: | :---: | :---:
__BAAI/bge-base-en-v1.5__ | 0.2568 | 0.4523 | 0.1305 | 0.2375
__BAI/bge-small-en__ | 0.2863 | 0.5116 | 0.161 | 0.3024


### Natural Language to SQL
Let's create a pipeline for the generation of SQL queries starting from natural language.

In [ ]:
!pip install -qU sqlalchemy

In [ ]:
from sqlalchemy import create_engine
from llama_index import SQLDatabase
from llama_index import ServiceContext
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine
from llama_index.tools.query_engine import QueryEngineTool

In [ ]:
DB_CSV_PATH = '/content/Seminar-Chatbot/BA_AirlineReviews.csv'

df = pd.read_csv(DB_CSV_PATH)
df = df.drop(columns=['Unnamed: 0']).dropna()
df.head()

,OverallRating,ReviewHeader,Name,Datetime,VerifiedReview,ReviewBody,TypeOfTraveller,SeatType,Route,DateFlown,SeatComfort,CabinStaffService,GroundService,ValueForMoney,Recommended,Aircraft,Food&Beverages,InflightEntertainment,Wifi&Connectivity
1,3.0,"""do not upgrade members based on status""",Austin Jones,19th November 2023,True,I recently had a delay on British Airways from...,Business,Economy Class,Brussels to London,November 2023,2.0,3.0,1.0,2.0,no,A320,1.0,2.0,2.0
8,2.0,"""Angry, disappointed, and unsatisfied""",Massimo Tricca,5th November 2023,False,"Angry, disappointed, and unsatisfied. My route...",Family Leisure,Economy Class,London Heatrow to Atlanta,November 2023,4.0,5.0,3.0,5.0,yes,Boeing 777,4.0,4.0,3.0
25,5.0,"""Club Europe is simply a joke""",M Dale,14th October 2023,True,I am a frequent flyer with BA and have been fo...,Business,Business Class,London to Istanbul,October 2023,3.0,4.0,3.0,2.0,no,A320,1.0,1.0,1.0
33,10.0,"""Excellent service levels""",Peter Costello,7th October 2023,True,"Excellent service levels, proactive crew and s...",Solo Leisure,First Class,London to New York JFK,October 2023,5.0,5.0,5.0,5.0,yes,Boeing 777,5.0,4.0,5.0
34,1.0,"""British Airways was absolutely shocking""",Kane Kelly,5th October 2023,False,Booked a very special holiday for me and my pa...,Couple Leisure,Business Class,Heathrow to Marseille,August 2023,1.0,1.0,1.0,1.0,no,BA366,1.0,1.0,1.0


Let's now create a SQL database from the Pandas DataFrame.

In [ ]:
engine = create_engine('sqlite://', echo=False)
df.to_sql(name='reviews', con=engine)
reviews_db = SQLDatabase(engine,include_tables=['reviews'])

We can now instantiate the LLM that will create queries and access this database.

In [ ]:
context_window = 8192
max_output_tokens = 256

prompt_helper_config = {
    'context_window' : context_window,
    'num_output' : max_output_tokens,
    'chunk_overlap_ratio' : 0 # The percentage token amount that each chunk should overlap.
}
prompt_helper = PromptHelper(**prompt_helper_config)


custom_model_config = {
    'verbose' : True,
    'temperature' : 0.2,
    'max_new_tokens' : max_output_tokens,
    'model_kwargs' : {
        "n_gpu_layers": 50, # map layer to GPU
        "device" : device,
        "stop" : ['[|Umano|]', '[|AI|]','[end of text]'],
    }
}

llm = LlamaCPP(
    model_path=zephyr_model_file,
    context_window=context_window,
    **custom_model_config
)
llm_predictor = LLMPredictor(llm=llm)

# Create Service Context Manager
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor,
                                               embed_model='local',
                                               prompt_helper=prompt_helper)

Once we have constructed our SQL database, we can use the `NLSQLTableQueryEngine` to construct natural language queries that are synthesized into SQL queries.


In [ ]:
sql_query_engine = NLSQLTableQueryEngine(sql_database=reviews_db,
                                         tables=['reviews'],
                                         service_context=service_context)

Let's define a prompt that can help the model understand the query for the translation of plain text into queries

In [ ]:
description = ("""
    Provides information about airlines reviews from reviews table.
    Use a detailed plain text question as input to the tool.
""")

QUEST = "What are the top 10 bad ReviewBody?"
INPUT = f"{description}\n{QUEST}"

response = sql_query_engine.query(INPUT)


llama_print_timings:        load time =     505.74 ms
llama_print_timings:      sample time =     144.83 ms /   256 runs   (    0.57 ms per token,  1767.63 tokens per second)
llama_print_timings: prompt eval time =     504.93 ms /   369 tokens (    1.37 ms per token,   730.80 tokens per second)
llama_print_timings:        eval time =    6715.01 ms /   255 runs   (   26.33 ms per token,    37.97 tokens per second)
llama_print_timings:       total time =    8230.78 ms /   624 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     505.74 ms
llama_print_timings:      sample time =     140.73 ms /   256 runs   (    0.55 ms per token,  1819.11 tokens per second)
llama_print_timings: prompt eval time =    1007.70 ms /   771 tokens (    1.31 ms per token,   765.11 tokens per second)
llama_print_timings:        eval time =    7214.47 ms /   255 runs   (   28.29 ms per token,    35.35 tokens per second)
llama_print_timings:       total time =    9289.95 ms /  1026 

In [ ]:
response_template = """
## Question

{question}

## Answer
```
{response}
```
## Generated SQL Query
```
{sql}
```
"""

response_md = str(response)
sql = response.metadata["sql_query"]


display(Markdown(response_template.format(
        question=QUEST,
        response=response_md,
        sql=sql,
    )))


## Question

What are the top 10 bad ReviewBody?

## Answer
```

The following are the top 10 worst ReviewBodies based on OverallRating:

1. "Excellent service levels, proactive crew and superb food and beverages. I found all aspects of the service to be superior to BA business class, even though the departure was delayed due to chaos among ground staff at Vancouver. The crew made the flight outstanding." (543 characters truncated) ... Ent, solid, comfortable service from start to finish and if the price was right, I would not hesitate to book First Class with British Airways again."

2. "This review is specifically about Maddie, crew member who took care of us. She was attentive, warm, friendly and nothing was too much trouble. Smiling from start to finish. She should be the brand ambassador for BA."

3. "Outstanding inflight service from the crew. Friendly, professional and helpful. The very best of British. Even though the departure was delayed due to chaos among ground staff at Vancouver, it didn’t matter. The crew made the flight outstanding."

4. "Check in at IAD was quick and easy. Lounge in Washington was a BA lounge and
```
## Generated SQL Query
```
SELECT ReviewBody FROM reviews ORDER BY OverallRating DESC LIMIT 10;
```


### RAG - Question-Answering
<a id='chandler_bing'></a>

> _Could we have any more files ??_

For this part we are going to use the Zephyr model.

In [ ]:
!pip install ctransformers>=0.2.24

In [ ]:
from ctransformers import AutoModelForCausalLM

zeph = AutoModelForCausalLM.from_pretrained("TheBloke/zephyr-7B-beta-GGUF", model_file=zephyr_model_file, gpu_layers=50)

In [ ]:
print(zeph("What's Chandler Bing's job?"))



That's a question that's been debated by "Friends" fans for years. Is he just a lucky trust fund kid who never has to work, or does he actually have a real job?

The answer is finally here, thanks to an interview with Matthew Perry, the actor who played Chandler on the hit '90s sitcom.

At Sunday night's Oscars, Perry was asked by E! News host Ryan Seacrest about what Chandler did for a living. And it turns out, we've been reading too much into it all these years.

"Chandler worked in advertising," Perry said. "He was a copywriter."

That's it. That's the answer. Chandler wasn't some kind of marketing whiz kid who became the CMO at a Fortune 500 company before turning 30, like Ross could have been. He wasn't an artist or musician struggling to make ends meet, like Joey and Rachel sometimes were.

He was just a regular guy trying to pay his rent with his writing skills, and we can all relate to that.


In [ ]:
MAX_NEW_TOKENS = 256

custom_model_config = {
    'verbose' : True,
    'temperature' : 0.0,
    'max_new_tokens' : MAX_NEW_TOKENS,
    'model_kwargs' : {
        "n_gpu_layers": 50,
        "device" : device,
        "stop" : ['[|Umano|]', '[|AI|]','[end of text]'],
    }
}

MAX_INPUT_SIZE = 4096
MAX_CHUNK_OVERLAP = 0 # Chunk overlap as a ratio of chunk size
prompt_helper = PromptHelper(MAX_INPUT_SIZE, MAX_NEW_TOKENS, MAX_CHUNK_OVERLAP)


llm = LlamaCPP(
    model_path=zephyr_model_file,
    context_window=8192,
    **custom_model_config
)

llm_predictor = LLMPredictor(llm=llm)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor,
                                               embed_model='local',
                                               prompt_helper=prompt_helper)

In [ ]:
print(zeph("What's Chandler Bing's job?"))



That's a question that's been debated by "Friends" fans for years.
He holds an assortment of office jobs and seemingly goes on interview after interview. There was also a season when he bought the Central Perk coffee house from Ross (David Schwimmer). But in the end, it's never really clear what Bing does for a living.
That might change if the show were to make a comeback, according to Matt LeBlanc. The actor, who played Joey Tribbiani on the hit NBC sitcom, said during an interview with The Daily Beast that he thinks Bing would be "rich as f--k" now.
LeBlanc's comments came after he was asked about a possible reunion of the cast and crew for the show's 25th anniversary. It would involve filming new scenes rather than pulling old episodes out of the archives. The series finale, which originally aired on May 6, 2004, wrapped up the six main characters' stories.
"It was so huge when we ended the show," LeBlanc said. "We'd have to be careful how we do it. I don'


Let's now implement a Retrieval-enhanced pipeline.

Using the `SimpleDirectoryReader` class we can index directly a whole directory or a subset of files. For the sake of saving some computational time, we are going to input a small subset of (long) text files.

In [ ]:
# Firsts 10 episodes of Season 1
reader = SimpleDirectoryReader(input_files=[f'./friends_scripts/01{i:02d}.txt' for i in range(1,10)])
documents = reader.load_data()

Thanks to the modularity of the LlamaIndex package, the rest of the pipeline remains unchanged.

In [ ]:
# Split data into chunks
node_parser = SentenceSplitter(chunk_size=256)
nodes = node_parser.get_nodes_from_documents(documents)

# Initialize node structure
vector_index = VectorStoreIndex(nodes,
                                service_context=service_context)

In [ ]:
# Create the retriever
retriever = vector_index.as_retriever(similarity_top_k=1)

With the following example we want to show how the RAG methodologies enforce the model to use the given context, limiting its previous knowledge. This is extremely important to limit hallucination. Indeed, if we trust the external data source, we can (almost) be sure that the model will not make up new and incorrect information.

In [ ]:
query_engine = vector_index.as_query_engine()

with suppress_stderr():
  out_gen = query_engine.query("What's Chandler Bing's job?")

In [ ]:
out_gen.response

" Chandler Bing's job is not explicitly mentioned in the provided context information. However, in the first context provided, Chandler mentions that if he doesn't input numbers, it doesn't make much of a difference, which could imply that he works with numbers or data in some capacity. Without further context or information, it's unclear what his specific job title or occupation might be."

## 3. Retrieval-augmented Chatbot with LlamaIndex
Finally we can implement our retrieval-based chatbot that leverages LlamaIndex to fetch data and a model of your choice as query engine.

LlamaIndex can be used as a support to retrieval and context management for a chatbot system. We can set a VectorIndex over a specific set of documents and employ different strategies to handle previous messages. Some of them are:
- `context`: parse context as a simple message exchange.
- `condense_question`: preprocess query and context history before running inference.

In [ ]:
from llama_index import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    ServiceContext,
    LLMPredictor,
    Response,
    PromptHelper
)
from llama_index.llms import LlamaCPP
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
import torch

device = "cuda" if torch.cuda.device_count() > 0 else "cpu"

llm = LlamaCPP(
    model_path='zephyr-7b-beta.Q4_K_M.gguf',
    verbose=True,
    temperature=0.0,
    max_new_tokens=256,
    context_window=4096,
    model_kwargs={
        "n_gpu_layers": 100,
        "stop" : ['user:', 'assistant:','[end of text]'],
        "device" : device
    }
)

In [ ]:
llm_predictor = LLMPredictor(llm=llm)

max_input_size = 1024
num_output = 256 # Number of outputs for the LLM.
max_chunk_overlap = 0 # Chunk overlap as a ratio of chunk size
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor,
                                               embed_model='local',
                                               prompt_helper=prompt_helper)

In [ ]:
from llama_index import set_global_service_context

# define the custom model as baseline for all inference operations
set_global_service_context(service_context)

# Define Retrieval space
data = SimpleDirectoryReader(input_files=['Seminar-Chatbot/ai_Leibniz_essey.txt']).load_data()

# Create an index of choice
index = VectorStoreIndex.from_documents(data,
                                        service_context=service_context)

In [ ]:
def start_chat(chat_engine):
  max_char_inline = 20
  while True:
    print("\033[96m[USER]:\x1b[0m", end=' ')
    user_input = input()
    if user_input == 'stop': break

    print("\033[92m[AI]:\x1b[0m", end=' ')

    with suppress_stderr():
      response = chat_engine.stream_chat(user_input)
      counter = 0
      for token in response.response_gen:
        counter += 1
        if token != '\n': print(token, end="")
        if counter > max_char_inline and not token.isalpha(): print(); counter = 0
      print()


In [ ]:
chat_engine = index.as_chat_engine(chat_mode="context")

start_chat(chat_engine)

[USER]: Why is Leibniz considered one of the fathers of AI?
[AI]: Leibniz is considered one of the fathers of AI because he proposed a plan for
 a universal language, an artificial language composed of symbols that would stand for concepts or ideas and logical rules for
 their valid manipulation. He believed that such a language would perfectly mirror the processes of intelligible human reasoning
. This plan has led some to believe that Leibniz came close to anticipating artificial intelligence.
 Additionally, Leibniz had a specific view about the nature of human cognitive processes, particularly about the
 nature of human reasoning, which is essentially symbolic and follows determinable axioms of logic. Reg
ardless of whether or not Leibniz should be seen as the grandfather of artificial intelligence, he did
 conceive of human cognition in essentially computational terms.
[USER]: What would Spinoza think about this?
[AI]: Spinoza, like Leibniz, had a specific view about the nature of 

In [ ]:
chat_engine = index.as_chat_engine(chat_mode="condense_question",
                                   verbose=True)

start_chat(chat_engine)

[USER]: When was Paul McCartney born?
[AI]: Querying with: What is the birthdate of Paul McCartney?
18-Jun-1942
[USER]: In which bands did he play?
[AI]: Querying with: What bands did Paul McCartney play in?
Paul McCartney was a member of The Beatles, which is the band he played in
.
[USER]: In which other bands did he play?
[AI]: Querying with: What other bands did Paul McCartney play in besides The Beatles?
<ul><li><a href="https://www.wingscentral.
com/" target="other">Wings</a></li><li><a href="https://
www.mccartney.com/music/solo-albums" target="other">
Solo albums by Paul McCartney</a></li></ul></ul>

[USER]: What about The Querryman?
[AI]: Querying with: Did Paul McCartney play in any other bands besides The Beatles and Wings, or did he release solo albums?
Paul McCartney played in several other bands besides The Beatles and Wings. He was a
 member of The Quarrymen, which later became The Beatles, from 1957 to
 1960. After The Beatles disbanded in 1970, he
 formed Wings with hi